In [ ]:
from __future__ import absolute_import, division, print_function
import os

PREF = '001a01a' 
KAGGLE = 'KAGGLE_WORKING_DIR' in os.environ
COLAB = 'COLAB_GPU' in os.environ

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import random
from multiprocessing import Pool, cpu_count
import tensorflow as tf
from keras import backend as K
import keras
from PIL import Image
from PIL import ImageOps
import cv2

import time

kernel_time = time.time()

def tic_tac(restart):
    global kernel_time 
    
    if restart:
        kernel_time = time.time()
    else:
        t_sec = round(time.time() - kernel_time)
        (t_min, t_sec) = divmod(t_sec,60)
        (t_hour,t_min) = divmod(t_min,60) 
        print('Time passed: {}hour:{}min:{}sec'.format(t_hour,t_min,t_sec))
    return

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
    tf.set_random_seed(seed)

tic_tac(True)
PREF

In [ ]:
if COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')

In [ ]:
#import keras
#from keras import backend as K

import tensorflow
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import os

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline


In [5]:
# from PIL import Image, ImageDraw, ImageFont, ImageOps
# fontsize = 50

# # From https://www.google.com/get/noto/
# !wget -q --show-progress https://noto-website-2.storage.googleapis.com/pkgs/NotoSansCJKjp-hinted.zip
# !unzip -p NotoSansCJKjp-hinted.zip NotoSansCJKjp-Regular.otf > NotoSansCJKjp-Regular.otf
# !rm NotoSansCJKjp-hinted.zip

# font = ImageFont.truetype('./NotoSansCJKjp-Regular.otf', fontsize, encoding='utf-8')

In [6]:
# !ls 

In [7]:
# !unzip darknet-master.zip


In [8]:
import os

if COLAB:
    os.chdir('/content/gdrive/My Drive/kuzushiji-recognition/keras-retinanet-master/')

In [9]:
import sys 
import os
sys.path.append('.')

In [10]:
!pip install --upgrade git+https://github.com/broadinstitute/keras-resnet
import keras
import keras_resnet

  Cloning https://github.com/broadinstitute/keras-resnet to /tmp/pip-req-build-f5_u_h6k
  Running command git clone -q https://github.com/broadinstitute/keras-resnet /tmp/pip-req-build-f5_u_h6k
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=22143 sha256=b9d6f72640bd212fb7d0e538055c2a529e2ac5f320413535944ea6042b50a26f
  Stored in directory: /tmp/pip-ephem-wheel-cache-iku1etau/wheels/10/52/f3/6a1fdbfb022ce9abfdf00a1ca7e90cef71dea99976edbcb53f
Successfully built keras-resnet
  Found existing installation: keras-resnet 0.2.0
    Uninstalling keras-resnet-0.2.0:
      Successfully uninstalled keras-resnet-0.2.0
  Rolling back uninstall of keras-resnet
  Moving to /home/maxim/.local/lib/python3.5/site-packages/keras_resnet-0.2.0.dist-info/
   from /home/maxim/.local/lib/python3.5/site-packages/~eras_resnet-0.2.0.dist-info
  Moving to /home/maxim/.local/lib/python3.5/site-packages/keras_resnet/
   from /home/maxim/.local/lib/python3.5/site-packages/~er

In [11]:
import argparse
import os
import sys
import warnings

import keras
import keras.preprocessing.image
import tensorflow as tf

# Allow relative imports when being executed as script.
# if __name__ == "__main__" and __package__ is None:
#     sys.path.insert(0, os.path.join(os.path.dirname(__file__), '..', '..'))
#     import keras_retinanet.bin  # noqa: F401
#     __package__ = "keras_retinanet.bin"

# Change these to absolute imports if you copy this script outside the keras_retinanet package.
import keras_retinanet.layers   # noqa: F401
import keras_retinanet.losses
import keras_retinanet.models as models
from keras_retinanet.callbacks import RedirectModel
from keras_retinanet.callbacks.eval import Evaluate
from keras_retinanet.models.retinanet import retinanet_bbox
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
from keras_retinanet.preprocessing.kitti import KittiGenerator
from keras_retinanet.preprocessing.open_images import OpenImagesGenerator
from keras_retinanet.preprocessing.pascal_voc import PascalVocGenerator
from keras_retinanet.utils.anchors import make_shapes_callback
from keras_retinanet.utils.config import read_config_file, parse_anchor_parameters
from keras_retinanet.utils.keras_version import check_keras_version
from keras_retinanet.utils.model import freeze as freeze_model
from keras_retinanet.utils.transform import random_transform_generator
from keras_retinanet.utils.image import random_visual_effect_generator


In [12]:
def makedirs(path):
    # Intended behavior: try to create the directory,
    # pass if the directory exists already, fails otherwise.
    # Meant for Python 2.7/3.n compatibility.
    try:
        os.makedirs(path)
    except OSError:
        if not os.path.isdir(path):
            raise


def get_session():
    """ Construct a modified tf session.
    """
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)


def model_with_weights(model, weights, skip_mismatch):
    """ Load weights for model.

    Args
        model         : The model to load weights for.
        weights       : The weights to load.
        skip_mismatch : If True, skips layers whose shape of weights doesn't match with the model.
    """
    if weights is not None:
        model.load_weights(weights, by_name=True, skip_mismatch=skip_mismatch)
    return model




In [13]:
def create_models(backbone_retinanet, num_classes, weights, multi_gpu=0,
                  freeze_backbone=False, lr=1e-5, config=None):
    """ Creates three models (model, training_model, prediction_model).

    Args
        backbone_retinanet : A function to call to create a retinanet model with a given backbone.
        num_classes        : The number of classes to train.
        weights            : The weights to load into the model.
        multi_gpu          : The number of GPUs to use for training.
        freeze_backbone    : If True, disables learning for the backbone.
        config             : Config parameters, None indicates the default configuration.

    Returns
        model            : The base model. This is also the model that is saved in snapshots.
        training_model   : The training model. If multi_gpu=0, this is identical to model.
        prediction_model : The model wrapped with utility functions to perform object detection (applies regression values and performs NMS).
    """

    modifier = freeze_model if freeze_backbone else None

    # load anchor parameters, or pass None (so that defaults will be used)
    anchor_params = None
    num_anchors   = None
    if config and 'anchor_parameters' in config:
        anchor_params = parse_anchor_parameters(config)
        num_anchors   = anchor_params.num_anchors()

    # Keras recommends initialising a multi-gpu model on the CPU to ease weight sharing, and to prevent OOM errors.
    # optionally wrap in a parallel model
    if multi_gpu > 1:
        from keras.utils import multi_gpu_model
        with tf.device('/cpu:0'):
            model = model_with_weights(backbone_retinanet(num_classes, num_anchors=num_anchors, modifier=modifier), weights=weights, skip_mismatch=True)
        training_model = multi_gpu_model(model, gpus=multi_gpu)
    else:
        model          = model_with_weights(backbone_retinanet(num_classes, num_anchors=num_anchors, modifier=modifier), weights=weights, skip_mismatch=True)
        training_model = model

    # make prediction model
    prediction_model = retinanet_bbox(model=model, anchor_params=anchor_params)

    # compile model
    training_model.compile(
        loss={
            'regression'    : losses.smooth_l1(),
            'classification': losses.focal()
        },
        optimizer=keras.optimizers.adam(lr=lr, clipnorm=0.001)
    )

    return model, training_model, prediction_model




In [14]:
def create_callbacks(model, training_model, prediction_model, validation_generator, 
    tensorboard_dir='./logs', 
    batch_size=1, 
    evaluation=True,
    snapshots=True,
    snapshot_path='./snapshot',
    backbone_name='resnet50',
    dataset_type='cvs'):
    """ Creates the callbacks to use during training.

    Args
        model: The base model.
        training_model: The model that is used for training.
        prediction_model: The model that should be used for validation.
        validation_generator: The generator for creating validation data.
        args: parseargs args object.

    Returns:
        A list of callbacks used for training.
    """
    callbacks = []

    tensorboard_callback = None

    if tensorboard_dir is not None:
        tensorboard_callback = keras.callbacks.TensorBoard(
            log_dir                = tensorboard_dir,
            histogram_freq         = 0,
            batch_size             = batch_size,
            write_graph            = True,
            write_grads            = False,
            write_images           = False,
            embeddings_freq        = 0,
            embeddings_layer_names = None,
            embeddings_metadata    = None
        )
        callbacks.append(tensorboard_callback)

    if evaluation and validation_generator:
        evaluation = Evaluate(validation_generator, tensorboard=tensorboard_callback, weighted_average=args.weighted_average)
        evaluation = RedirectModel(evaluation, prediction_model)
        callbacks.append(evaluation)

    # save the model
    if snapshots:
        # ensure directory created first; otherwise h5py will error after epoch. 
        makedirs(snapshot_path)
        checkpoint = keras.callbacks.ModelCheckpoint(
            os.path.join(
                snapshot_path,
                '{backbone}_{dataset_type}_{{epoch:02d}}.h5'.format(backbone=backbone_name, dataset_type=dataset_type)
            ),
            verbose=1,
            # save_best_only=True,
            # monitor="mAP",
            # mode='max'
        )
        checkpoint = RedirectModel(checkpoint, model)
        callbacks.append(checkpoint)

    callbacks.append(keras.callbacks.ReduceLROnPlateau(
        monitor    = 'loss',
        factor     = 0.1,
        patience   = 2,
        verbose    = 1,
        mode       = 'auto',
        min_delta  = 0.0001,
        cooldown   = 0,
        min_lr     = 0
    ))

    return callbacks


In [15]:
def create_generators(preprocess_image, batch_size=1, image_min_side=640, image_max_side=1000,
    train_annotations='./data/kuzushiji-recognition/060a01d_train.txt',
    train_classes='./data/kuzushiji-recognition/060a01d__kuzushiji.labels',
    val_annotations='./data/kuzushiji-recognition/060a01d_test.txt'):
    """ Create generators for training and validation.

    Args
        args             : parseargs object containing configuration for generators.
        preprocess_image : Function that preprocesses an image for the network.
    """
    common_args = {
        'batch_size'       : batch_size,
        #'config'           : args.config,
        'image_min_side'   : image_min_side,
        'image_max_side'   : image_max_side,
        'preprocess_image' : preprocess_image,
    }

    # create random transform generator for augmenting training data
    if False: #args.random_transform:
        transform_generator = random_transform_generator(
            min_rotation=-0.1,
            max_rotation=0.1,
            min_translation=(-0.1, -0.1),
            max_translation=(0.1, 0.1),
            min_shear=-0.1,
            max_shear=0.1,
            min_scaling=(0.9, 0.9),
            max_scaling=(1.1, 1.1),
            flip_x_chance=0.5,
            flip_y_chance=0.5,
        )
        visual_effect_generator = random_visual_effect_generator(
            contrast_range=(0.9, 1.1),
            brightness_range=(-.1, .1),
            hue_range=(-0.05, 0.05),
            saturation_range=(0.95, 1.05)
        )
    else:
        transform_generator = random_transform_generator(flip_x_chance=0.5)
        visual_effect_generator = None

    # Dataset type 'csv':
    train_generator = CSVGenerator(
        train_annotations,
        train_classes,
        transform_generator=transform_generator,
        visual_effect_generator=visual_effect_generator,
        **common_args
    )

    if val_annotations is not None:
        validation_generator = CSVGenerator(
            val_annotations,
            train_classes,
            shuffle_groups=False,
            **common_args
        )
    else:
        validation_generator = None
    return train_generator, validation_generator


In [ ]:
#args = "-u keras_retinanet/bin/train.py --image-min-side 640 --image-max-side 1000 --weights ./snapshots/resnet50_csv_05.h5 csv ./data/kuzushiji-recognition/060a01d_train.txt ./data/kuzushiji-recognition/060a01d__kuzushiji.labels --val-annotations ./data/kuzushiji-recognition/060a01d_test.txt"
backbone_name='resnet50'
snapshot = './snapshots/resnet50_csv_05.h5'
weights = None
multi_gpu = 0
#
epochs=50
batch_size=1
steps=3600
snapshot_path='./snapshots'
tensorboard_dir='./logs'
no_snapshots=False
no_evaluation=False
freeze_backbone=False  ##
random_transform=True
#config='../*.ini'
weighted_average=True
compute_val_loss=True
multiprocessing=True
workers=1
max_queue_size=10
lr = 1e-5

# create object that stores backbone information
backbone = models.backbone(backbone_name)

# make sure keras is the minimum required version
check_keras_version()

# optionally choose specific GPU
if COLAB:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
keras.backend.tensorflow_backend.set_session(get_session())

# create the generators
train_generator, validation_generator = create_generators(backbone.preprocess_image)

# create the model
if snapshot is not None:
    print('Loading model, this may take a second...')
    model            = models.load_model(snapshot, backbone_name=backbone_name)
    training_model   = model
    anchor_params    = None
    prediction_model = retinanet_bbox(model=model, anchor_params=anchor_params)
else:
    weights = weights
    # default to imagenet if nothing else is specified
    if weights is None and args.imagenet_weights:
        weights = backbone.download_imagenet()

    print('Creating model, this may take a second...')
    model, training_model, prediction_model = create_models(
        backbone_retinanet=backbone.retinanet,
        num_classes=train_generator.num_classes(),
        weights=weights,
        multi_gpu=multi_gpu,
        freeze_backbone=freeze_backbone,
        lr=lr,
        config=args.config
    )

# print model summary
print(model.summary())

# this lets the generator compute backbone layer shapes using the actual backbone model
if 'vgg' in backbone_name or 'densenet' in backbone_name:
    train_generator.compute_shapes = make_shapes_callback(model)
    if validation_generator:
        validation_generator.compute_shapes = train_generator.compute_shapes

# create the callbacks
callbacks = create_callbacks(
    model,
    training_model,
    prediction_model,
    validation_generator,
    
    tensorboard_dir=tensorboard_dir, 
    batch_size=batch_size, 
    evaluation=not no_evaluation,
    snapshots=not no_snapshots,
    snapshot_path=snapshot_path,
    backbone_name=backbone_name,
    dataset_type='cvs'
)

if not compute_val_loss:
    validation_generator = None

# start training
history = training_model.fit_generator(
    generator=train_generator,
    steps_per_epoch=steps,
    epochs=epochs,
    verbose=1,
    callbacks=callbacks,
    workers=workers,
    use_multiprocessing=multiprocessing,
    max_queue_size=max_queue_size,
    validation_data=validation_generator
)


W0921 20:39:23.765415 139816213817088 deprecation_wrapper.py:119] From /home/maxim/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0921 20:39:23.788585 139816213817088 deprecation_wrapper.py:119] From /home/maxim/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0921 20:39:23.885725 139816213817088 deprecation_wrapper.py:119] From /home/maxim/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0921 20:39:23.903318 139816213817088 deprecation_wrapper.py:119] From /home/maxim/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:1919: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0921 20:39:23.

Loading model, this may take a second...


W0921 20:39:24.891520 139816213817088 deprecation_wrapper.py:119] From /home/maxim/Documents/keras-retinanet-master/keras_retinanet/backend/tensorflow_backend.py:68: The name tf.image.resize_images is deprecated. Please use tf.image.resize instead.

W0921 20:39:25.730221 139816213817088 deprecation_wrapper.py:119] From /home/maxim/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0921 20:39:27.051978 139816213817088 deprecation_wrapper.py:119] From /home/maxim/.local/lib/python3.5/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0921 20:39:27.061273 139816213817088 deprecation.py:323] From /home/maxim/Documents/keras-retinanet-master/keras_retinanet/backend/tensorflow_backend.py:104: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
padding_conv1 (ZeroPadding2D)   (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9408        padding_conv1[0][0]              
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
conv1_relu

W0921 20:39:34.640139 139816213817088 deprecation_wrapper.py:119] From /home/maxim/.local/lib/python3.5/site-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0921 20:39:34.640919 139816213817088 deprecation_wrapper.py:119] From /home/maxim/.local/lib/python3.5/site-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/50
1078/3600 [=======>......................] - ETA: 3:07:38 - loss: 0.3883 - regression_loss: 0.3448 - classification_loss: 0.0435

Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


In [ ]:
tic_tac(False)

In [ ]:
# EOF